# Lattice PDF

In this notebook, we will use the architecture developed previously to variationally find ground hadron states, $|P\rangle$ and use these to generate the parton distribution function, $f_{q/h}(x)$

The lattice PDF is given as: $$f_{q/h}(x) = \int dz e^{ixP^+z}\zeta(z)$$ with $$\zeta(z) = \langle P| \left(\sum_{i,j = 0}^1 e^{iHt}\chi^\dagger_{2(z + i)}e^{-iHt}\chi_{2j + 1} \right) |P\rangle $$

We utilize lightcone coordinates, so to keep this correlation funciton on the lightcone, we set $t = z$. $z$ represents the physical continuum coordinate, i.e. $z\in\mathbb{R}$ Mapping this to the lattice, we have to double the number of lattice points for each physical coordinate (to utilize the staggered fermion approach of ADD CITATION), i.e. $N = 2Z$ (capital letters correspond to cutoffs, $0 \leq z \leq Z$). For each $N$, we have to use $2N$ qubits to distinguish the two colors of red and green. 

In [36]:
from symmer import PauliwordOp, QuantumState
import qiskit
from hamiltonian import H
from basis_states import *
from openfermion import *

In [2]:
N = 4

In [3]:
full_ham = H(N)

In [4]:
mesons, baryons = get_basis(N)

In [5]:
oh_bin_dict = one_hot_to_binary_encoding(mesons)

In [8]:
Ham_meson = H_sector(full_ham, mesons)

Building operator via projectors diag elements:   0%|          | 0/35 [00:00<?, ?it/s]

Building operator via projectors off-diag elements:   0%|          | 0/168 [00:00<?, ?it/s]

In [9]:
from symmer.utils import exact_gs_energy

gs_energy, psi = exact_gs_energy(Ham_meson.to_sparse_matrix)

In [31]:
def revert_to_full_basis(state, encoding_dictionary, number_full_qubits):
    psi_OH = QuantumState([0] * number_full_qubits)*0

    for psi_state in psi.to_dictionary:
        psi_OH += encoding_dictionary[psi_state] * state.to_dictionary[psi_state]
    
    return psi_OH

In [33]:
full_psi = revert_to_full_basis(psi, oh_bin_dict, full_ham.n_qubits)
full_psi.sort('decreasing').cleanup(1e-3)

-0.027-0.028j |00001111> +
 0.121+0.125j |00011011> +
-0.024-0.024j |00011110> +
-0.121-0.125j |00100111> +
 0.024+0.024j |00101101> +
-0.636-0.655j |00110011> +
 0.125+0.128j |00110110> +
-0.125-0.128j |00111001> +
-0.028-0.029j |00111100> +
 0.017+0.017j |01001011> +
-0.003-0.003j |01001110> +
 0.088+0.091j |01100011> +
-0.018-0.018j |01100110> +
 0.017+0.018j |01101001> +
 0.004+0.004j |01101100> +
-0.002-0.002j |01110010> +
-0.017-0.017j |10000111> +
 0.003+0.003j |10001101> +
-0.088-0.091j |10010011> +
 0.017+0.018j |10010110> +
-0.018-0.018j |10011001> +
-0.004-0.004j |10011100> +
 0.002+0.002j |10110001> +
-0.020-0.021j |11000011> +
 0.004+0.004j |11000110> +
-0.004-0.004j |11001001> +
-0.001-0.001j |11001100>

In [52]:
full_ham.n_qubits

8

In [50]:
of_op = jordan_wigner(FermionOperator('3^'))

In [51]:
PauliwordOp.from_openfermion(of_op)

 0.500+0.000j ZZZX +
 0.000-0.500j ZZZY

In [72]:
def chis(z, num_full_qubits):
    chi_pairs = []
    
    for i in range(0, 2):
        for j in range(0, 2):
            qubit_string_create = str(2 * (z + i)) + "^"
            qubit_string_annihilate = str(2 * j + 1)
            pauli_create = PauliwordOp.from_openfermion(jordan_wigner(FermionOperator(qubit_string_create)))
            pauli_annihilate = PauliwordOp.from_openfermion(jordan_wigner(FermionOperator(qubit_string_annihilate)))

            if pauli_create.n_qubits < num_full_qubits:
                missing_qubits_create = num_full_qubits - pauli_create.n_qubits
                pauli_create = \
                    pauli_create.tensor(PauliwordOp.from_dictionary({'I'*missing_qubits_create: 1.0}))
            if pauli_annihilate.n_qubits < num_full_qubits:
                missing_qubits_annihilate = num_full_qubits - pauli_annihilate.n_qubits
                pauli_annihilate = \
                    pauli_annihilate.tensor(PauliwordOp.from_dictionary({'I'*missing_qubits_annihilate: 1.0}))
            chi_pairs.append(((i,j), pauli_create, pauli_annihilate))
    return chi_pairs

In [79]:
z = 1

In [80]:
chis(z, full_ham.n_qubits)

[((0, 0),
   0.500+0.000j ZZXIIIII +
   0.000-0.500j ZZYIIIII,
   0.500+0.000j ZXIIIIII +
   0.000+0.500j ZYIIIIII),
 ((0, 1),
   0.500+0.000j ZZXIIIII +
   0.000-0.500j ZZYIIIII,
   0.500+0.000j ZZZXIIII +
   0.000+0.500j ZZZYIIII),
 ((1, 0),
   0.500+0.000j ZZZZXIII +
   0.000-0.500j ZZZZYIII,
   0.500+0.000j ZXIIIIII +
   0.000+0.500j ZYIIIIII),
 ((1, 1),
   0.500+0.000j ZZZZXIII +
   0.000-0.500j ZZZZYIII,
   0.500+0.000j ZZZXIIII +
   0.000+0.500j ZZZYIIII)]

In [83]:
from symmer.evolution import trotter

In [85]:
exp_op_plus = full_ham.multiply_by_constant(1j * z)
exp_op_minus = full_ham.multiply_by_constant(-1j * z)
trotter(exp_op_plus), trotter(exp_op_minus)

( 0.070-0.010j IIIIIIII +
  0.005+0.035j IIIIIIIZ +
  0.005+0.035j IIIIIIZI +
 -0.018+0.002j IIIIIIZZ +
 -0.007-0.034j IIIIIZII +
  0.025-0.004j IIIIIZIZ +
  0.018-0.004j IIIIIZZI +
  0.003+0.013j IIIIIZZZ +
 -0.007-0.034j IIIIZIII +
  0.018-0.004j IIIIZIIZ +
  0.025-0.004j IIIIZIZI +
  0.003+0.013j IIIIZIZZ +
 -0.006-0.007j IIIIZZII +
  0.003-0.006j IIIIZZIZ +
  0.003-0.006j IIIIZZZI +
  0.006+0.001j IIIIZZZZ +
  0.018+0.059j IIIZIIII +
 -0.030+0.009j IIIZIIIZ +
 -0.029+0.007j IIIZIIZI +
 -0.004-0.015j IIIZIIZZ +
  0.027-0.010j IIIZIZII +
  0.007+0.020j IIIZIZIZ +
  0.006+0.013j IIIZIZZI +
 -0.010+0.004j IIIZIZZZ +
  0.037-0.028j IIIZZIII +
  0.016+0.019j IIIZZIIZ +
  0.017+0.025j IIIZZIZI +
 -0.013+0.009j IIIZZIZZ +
  0.004-0.017j IIIZZZII +
  0.013-0.000j IIIZZZIZ +
  0.012+0.001j IIIZZZZI +
  0.001+0.008j IIIZZZZZ +
  0.018+0.059j IIZIIIII +
 -0.029+0.007j IIZIIIIZ +
 -0.030+0.009j IIZIIIZI +
 -0.004-0.015j IIZIIIZZ +
  0.037-0.028j IIZIIZII +
  0.017+0.025j IIZIIZIZ +
  0.016+0.01